In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
from dateutil.parser import isoparse
from datetime import datetime, timedelta
from DirectoryGenerator import DirectoryGenerator

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
totalDays = abs(endDate - startDate).days + 1
totalDays

In [ ]:
def parseTimestamp(timestamp):
    try:
        return isoparse(timestamp).astimezone(pytz.utc)
    except:
        return np.nan

In [ ]:
assignmentsDataSubmissionsFilesPath = str(input())

In [ ]:
assignmentsDataSubmissionsFiles = glob.glob(assignmentsDataSubmissionsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsDataSubmissionsData = pd.DataFrame()
for assignmentsDataSubmissionsFile in assignmentsDataSubmissionsFiles:
    assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.append(pd.read_csv(assignmentsDataSubmissionsFile), ignore_index = True)
    
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData[['metadata_event_time', 'metadata_context_id', 'metadata_context_role', 'metadata_user_id', 'body_asset_id']]
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData['metadata_event_time'] = assignmentsDataSubmissionsData['metadata_event_time'].apply(parseTimestamp)
assignmentsDataSubmissionsData.dropna(subset=['metadata_event_time'], inplace = True)
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[(assignmentsDataSubmissionsData['metadata_event_time'] >= startDate) & (assignmentsDataSubmissionsData['metadata_event_time'] <= endDate)]
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[assignmentsDataSubmissionsData['metadata_context_role'] == 'StudentEnrollment']
assignmentsDataSubmissionsData.drop(['metadata_context_role'], axis = 1, inplace = True)
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData['metadata_event_time'] = assignmentsDataSubmissionsData['metadata_event_time'].apply(str)
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.groupby(['metadata_context_id', 'metadata_user_id'])['metadata_event_time'].apply(list).reset_index()
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData.to_csv("CourseLessonAccesses_Intercession_2022.csv", header = True, index = False)